In [17]:
print("Hello World")

Hello World


In [18]:
# Label Propagation Regression
from LabelPropagationRegression import *

In [19]:
import sklearn as sk
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.metrics import pairwise_distances
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.impute import SimpleImputer

In [4]:
data_c = pd.DataFrame({"col1":[1, 2, 3], "col2":[2,2,2],"col3":[3,3,3]})

data_c

,col1,col2,col3
0,1,2,3
1,2,2,3
2,3,2,3


In [5]:
len(data_c)

3

In [6]:
data_m = pd.DataFrame({"col1":[1, 2, 3], "col2":[2,2,np.nan], "col3":[3,3,3]})

data_m

,col1,col2,col3
0,1,2.0,3
1,2,2.0,3
2,3,NaN,3


In [7]:
data_x = pd.DataFrame({"col1":[1, 2, 3,4, 5, 6, 7, 8, 9, 10,np.nan], "col2":[3,3,3,3,3,3,3,3,3,3,np.nan], "col3":[np.nan,2,2,2,2,2,2,2,2,2,2]})

data_x

,col1,col2,col3
0,1.0,3.0,NaN
1,2.0,3.0,2.0
2,3.0,3.0,2.0
3,4.0,3.0,2.0
4,5.0,3.0,2.0
5,6.0,3.0,2.0
6,7.0,3.0,2.0
7,8.0,3.0,2.0
8,9.0,3.0,2.0
9,10.0,3.0,2.0


In [8]:
array = data_x.to_numpy()

array

array([[ 1.,  3., nan],
       [ 2.,  3.,  2.],
       [ 3.,  3.,  2.],
       [ 4.,  3.,  2.],
       [ 5.,  3.,  2.],
       [ 6.,  3.,  2.],
       [ 7.,  3.,  2.],
       [ 8.,  3.,  2.],
       [ 9.,  3.,  2.],
       [10.,  3.,  2.],
       [nan, nan,  2.]])

In [9]:
len(array)

11

In [10]:
# produzindo Iris com missing

iris = datasets.load_iris()

data = iris.data

Y=data[:,3]

M = np.random.randint(2, size=Y.shape[0]) # simulando que tem missing nos que estão com índice 1

Y_orig=data[:,3] # coluna que vai ter missing no meu exemplo

dataMiss = np.zeros(Y_orig.shape[0])
dataMiss[M==0] = Y_orig[M==0] # simulando um random missing data, quem é 1 vira missing, quem não é fica igual ao original
dataMiss[M==1] = np.nan

data = pd.DataFrame(data)
data = data[[0,1,2]]
data[3] = dataMiss
data = data.to_numpy()
len(data)
data




array([[5.1, 3.5, 1.4, nan],
       [4.9, 3. , 1.4, nan],
       [4.7, 3.2, 1.3, nan],
       [4.6, 3.1, 1.5, nan],
       [5. , 3.6, 1.4, nan],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, nan],
       [4.4, 2.9, 1.4, nan],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, nan],
       [4.8, 3.4, 1.6, nan],
       [4.8, 3. , 1.4, nan],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, nan],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, nan],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, nan],
       [5.4, 3.4, 1.7, nan],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, nan],
       [5. , 3.4, 1.6, nan],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, nan],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, nan],
       [5.2, 4.1, 1.5, nan],
       [5.5, 4.2, 1.4, nan],
       [4.9, 3

In [11]:
target = iris.target

target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [12]:
type(data)

numpy.ndarray

In [13]:
imputer = LabelPropagationRegression1(data_c, iteracoes=1000, epsilon=1e-32)

max interações:  1000
epsilon:  1e-32


In [14]:
model = imputer.fit(data, target)

Tipos de dado:  <class 'pandas.core.frame.DataFrame'>
      0    1    2    3  grupo
0   5.1  3.5  1.4  NaN      0
1   4.9  3.0  1.4  NaN      0
2   4.7  3.2  1.3  NaN      0
3   4.6  3.1  1.5  NaN      0
4   5.0  3.6  1.4  NaN      0
5   5.4  3.9  1.7  0.4      0
6   4.6  3.4  1.4  0.3      0
7   5.0  3.4  1.5  NaN      0
8   4.4  2.9  1.4  NaN      0
9   4.9  3.1  1.5  0.1      0
10  5.4  3.7  1.5  NaN      0
11  4.8  3.4  1.6  NaN      0
12  4.8  3.0  1.4  NaN      0
13  4.3  3.0  1.1  0.1      0
14  5.8  4.0  1.2  NaN      0
15  5.7  4.4  1.5  0.4      0
16  5.4  3.9  1.3  0.4      0
17  5.1  3.5  1.4  NaN      0
18  5.7  3.8  1.7  0.3      0
19  5.1  3.8  1.5  NaN      0
20  5.4  3.4  1.7  NaN      0
21  5.1  3.7  1.5  0.4      0
22  4.6  3.6  1.0  0.2      0
23  5.1  3.3  1.7  0.5      0
24  4.8  3.4  1.9  0.2      0
25  5.0  3.0  1.6  NaN      0
26  5.0  3.4  1.6  NaN      0
27  5.2  3.5  1.5  0.2      0
28  5.2  3.4  1.4  0.2      0
29  4.7  3.2  1.6  NaN      0
30  4.8  3.1  1.

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.

In [ ]:
x = model.transform(data)

2088.0465753424655
17.435983050999056
8.347508084791674
4.0274105361893575
1.966686152078827
0.9780711429204081
0.49796624882048857
0.2617057247878132
0.14098821018027996
0.07892601599428162
0.045812189648659074
0.026851403804137863
0.01564890708936617
0.009080570152361411
0.005251576320345452
0.0030292912137234174
0.0017438780145266897
0.0010023190401103887
0.0005753847458174199
0.000329980279551112
0.00018909695106616997
0.00010829743635798472
6.199320531752317e-05
3.547363548372928e-05
2.0292578996494726e-05
1.1605551353377486e-05
6.6360962880374075e-06
3.7939785276774884e-06
2.1688311638845548e-06
1.2396980543227798e-06
7.085553960228808e-07
4.0495436148813013e-07
2.3142912897178292e-07
1.3225553185947092e-07
7.557825021375919e-08
4.3188656095871636e-08
2.4679390753234998e-08
1.4102391698855854e-08
8.058347211736816e-09
4.6046386703579856e-09
2.6311249845534235e-09
1.503432822680395e-09
8.590682609721512e-10
4.908683459703411e-10
2.804795284916395e-10
1.602714627679802e-10
9.157596

In [ ]:
x 

array([[5.1       , 3.5       , 1.4       , 0.57878183],
       [4.9       , 3.        , 1.4       , 0.58498955],
       [4.7       , 3.2       , 1.3       , 0.55847806],
       [4.6       , 3.1       , 1.5       , 0.2       ],
       [5.        , 3.6       , 1.4       , 0.2       ],
       [5.4       , 3.9       , 1.7       , 0.62789741],
       [4.6       , 3.4       , 1.4       , 0.3       ],
       [5.        , 3.4       , 1.5       , 0.2       ],
       [4.4       , 2.9       , 1.4       , 0.2       ],
       [4.9       , 3.1       , 1.5       , 0.1       ],
       [5.4       , 3.7       , 1.5       , 0.60102024],
       [4.8       , 3.4       , 1.6       , 0.5972786 ],
       [4.8       , 3.        , 1.4       , 0.58063117],
       [4.3       , 3.        , 1.1       , 0.53001105],
       [5.8       , 4.        , 1.2       , 0.56940366],
       [5.7       , 4.4       , 1.5       , 0.59422009],
       [5.4       , 3.9       , 1.3       , 0.4       ],
       [5.1       , 3.5       ,

In [ ]:
data2 = pd.DataFrame({"col1":[1, 1, 1, 1, 1,1,1], "col2":[2,3,np.nan,3,2,3,np.nan], "col3":[np.nan,2,2,2,2,2,2]})

data2

,col1,col2,col3
0,1,2.0,NaN
1,1,3.0,2.0
2,1,NaN,2.0
3,1,3.0,2.0
4,1,2.0,2.0
5,1,3.0,2.0
6,1,NaN,2.0


In [ ]:
dataImp = model.transform(data2)
dataImp

39.2


TypeError: NumPy boolean array indexing assignment requires a 0 or 1-dimensional input, input has 2 dimensions

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.cluster import KMeans

# Selecionando os dados do iris
df = sns.load_dataset("iris")


# Nome das espécies de flores
list(df['species'].unique())

X = df.drop(columns='species').copy()

X


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
kmeans = KMeans(n_clusters = 3)
kmeans.fit(X)



/home/filipe/.local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=3)

In [ ]:
# Centroides das entradas
kmeans.cluster_centers_



array([[5.9016129 , 2.7483871 , 4.39354839, 1.43387097],
       [5.006     , 3.428     , 1.462     , 0.246     ],
       [6.85      , 3.07368421, 5.74210526, 2.07105263]])

In [ ]:
# Clusters das entradas
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0], dtype=int32)

In [ ]:
x = [0, 1, 2,10]

tamanho_x = len(x)

for i in range(tamanho_x):
    print(i)

0
1
2
3


In [ ]:
# Validação da divisão dos clusters
X_valida = X.copy()
df_valida = df.copy()
for i in range(2, 6):
    kmeans_valida = KMeans(n_clusters = i)
    kmeans_valida.fit(X_valida)
    df_valida['kmeans'] = kmeans_valida.labels_
    print('K =', i)
    for j in list(df['species'].unique()):
        per_valida = df_valida.query('species == @j')['kmeans'].value_counts().max()/df_valida.query('species == @j').shape[0]
        class_valida = df_valida.query('species == @j')['kmeans'].value_counts().index[0]
        print('Espécie {}'.format(j), 'Cluster', class_valida, '- Percentual de acerto {:.0%}'.format(per_valida))
    print(55*'=')

/home/filipe/.local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


K = 2
Espécie setosa Cluster 0 - Percentual de acerto 100%
Espécie versicolor Cluster 1 - Percentual de acerto 94%
Espécie virginica Cluster 1 - Percentual de acerto 100%
K = 3
Espécie setosa Cluster 0 - Percentual de acerto 100%
Espécie versicolor Cluster 1 - Percentual de acerto 96%
Espécie virginica Cluster 2 - Percentual de acerto 72%
K = 4
Espécie setosa Cluster 0 - Percentual de acerto 100%
Espécie versicolor Cluster 3 - Percentual de acerto 54%
Espécie virginica Cluster 2 - Percentual de acerto 64%
K = 5
Espécie setosa Cluster 1 - Percentual de acerto 100%
Espécie versicolor Cluster 0 - Percentual de acerto 52%
Espécie virginica Cluster 2 - Percentual de acerto 48%


/home/filipe/.local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/filipe/.local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/filipe/.local/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
df.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [ ]:
df["species"].describe()

count        150
unique         3
top       setosa
freq          50
Name: species, dtype: object

In [ ]:
len(df[df["species"]=='setosa'])

50

In [ ]:
len(df[df["species"]=='versicolor'])

50

In [ ]:
len(df[df["species"]=='virginica'])

50

In [ ]:
x = df[["sepal_length","sepal_width","petal_length","petal_width"]]

In [ ]:
X = StandardScaler().fit_transform(x)

X

array([[-0.90068117,  1.01900435, -1.34022653, -1.32726774],
       [-1.14301691, -0.13197948, -1.34022653, -1.33222597],
       [-1.38535265,  0.32841405, -1.39706395, -1.33222597],
       [-1.50652052,  0.09821729, -1.2833891 , -1.33222597],
       [-1.02184904,  1.24920112, -1.34022653, -1.33222597],
       [-0.53717756,  1.93979142, -1.16971425, -1.13714612],
       [-1.50652052,  0.78880759, -1.34022653, -1.20031776],
       [-1.02184904,  0.78880759, -1.2833891 , -1.33222597],
       [-1.74885626, -0.36217625, -1.34022653, -1.27833096],
       [-1.14301691,  0.09821729, -1.2833891 , -1.46413417],
       [-0.53717756,  1.47939788, -1.2833891 , -1.29119153],
       [-1.26418478,  0.78880759, -1.22655167, -1.15769677],
       [-1.26418478, -0.13197948, -1.34022653, -1.33716309],
       [-1.87002413, -0.13197948, -1.51073881, -1.46413417],
       [-0.05250608,  2.16998818, -1.45390138, -1.33222597],
       [-0.17367395,  3.09077525, -1.2833891 , -1.25587526],
       [-0.53717756,  1.

In [ ]:
import numpy as np

from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.5, min_samples=10).fit(X)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 4
Estimated number of noise points: 68


In [ ]:
from sklearn.cluster import MeanShift
import numpy as np


clustering = MeanShift().fit(X)
clustering.labels_



array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from sklearn.cluster import HDBSCAN

hdb = HDBSCAN()
hdb.fit(X)
hdb.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [ ]:
X

array([[-0.90068117,  1.01900435, -1.34022653, -1.32726774],
       [-1.14301691, -0.13197948, -1.34022653, -1.33222597],
       [-1.38535265,  0.32841405, -1.39706395, -1.33222597],
       [-1.50652052,  0.09821729, -1.2833891 , -1.33222597],
       [-1.02184904,  1.24920112, -1.34022653, -1.33222597],
       [-0.53717756,  1.93979142, -1.16971425, -1.13714612],
       [-1.50652052,  0.78880759, -1.34022653, -1.20031776],
       [-1.02184904,  0.78880759, -1.2833891 , -1.33222597],
       [-1.74885626, -0.36217625, -1.34022653, -1.27833096],
       [-1.14301691,  0.09821729, -1.2833891 , -1.46413417],
       [-0.53717756,  1.47939788, -1.2833891 , -1.29119153],
       [-1.26418478,  0.78880759, -1.22655167, -1.15769677],
       [-1.26418478, -0.13197948, -1.34022653, -1.33716309],
       [-1.87002413, -0.13197948, -1.51073881, -1.46413417],
       [-0.05250608,  2.16998818, -1.45390138, -1.33222597],
       [-0.17367395,  3.09077525, -1.2833891 , -1.25587526],
       [-0.53717756,  1.

In [ ]:
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

clustering = AgglomerativeClustering()


# Ajustar o modelo aos dados
result = clustering.fit_predict(X)

result




array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
result

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
x["agrupamento"] = result

x["teste"] = np.nan

x




,sepal_length,sepal_width,petal_length,petal_width,agrupamento,teste
0,5.1,3.5,1.4,0.2,1,NaN
1,4.9,3.0,1.4,0.2,1,NaN
2,4.7,3.2,1.3,0.2,1,NaN
3,4.6,3.1,1.5,0.2,1,NaN
4,5.0,3.6,1.4,0.2,1,NaN
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,NaN
146,6.3,2.5,5.0,1.9,0,NaN
147,6.5,3.0,5.2,2.0,0,NaN
148,6.2,3.4,5.4,2.3,0,NaN


In [ ]:
x.loc[x.agrupamento == 1, "teste"] = "success"

In [ ]:
x

,sepal_length,sepal_width,petal_length,petal_width,agrupamento,teste
0,5.1,3.5,1.4,0.2,1,success
1,4.9,3.0,1.4,0.2,1,success
2,4.7,3.2,1.3,0.2,1,success
3,4.6,3.1,1.5,0.2,1,success
4,5.0,3.6,1.4,0.2,1,success
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,NaN
146,6.3,2.5,5.0,1.9,0,NaN
147,6.5,3.0,5.2,2.0,0,NaN
148,6.2,3.4,5.4,2.3,0,NaN


In [ ]:
x.columns[:-1]

Index([0, 1, 2, 3], dtype='object')

In [ ]:

# Defina a função
def my_function(df, classe, colunas):
    for coluna in colunas:
        df.loc[df['classe'] == classe, coluna] = df.loc[df['classe'] == classe, coluna].apply(lambda x: funcao(x))

# Selecione os elementos da classe
df_a = df.loc[df['classe'] == 'A']

# Aplique a função
my_function(df_a, 'A', ['valor1', 'valor2'])

# Modifique os elementos da classe A
df.loc[df['classe'] == 'A', ['valor1', 'valor2']] = df_a[['valor1', 'valor2']]

# Imprimir o dataframe
print(df)

In [ ]:
def remover_outliers_iqr(df):
  """
  Remove outliers em cada coluna de um dataframe Pandas utilizando o IQR.

  Argumentos:
    df: Dataframe Pandas.

  Retorna:
    Dataframe Pandas com outliers substituidos por np.nan.
  """

  for coluna in df.columns:
    # Cálculo do primeiro e terceiro quartil
    q1 = df[coluna].quantile(0.25)
    q3 = df[coluna].quantile(0.75)

    # Cálculo do IQR
    iqr = q3 - q1

    # Definição dos limites inferior e superior
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    # Substituição de outliers por np.nan
    df.loc[df[coluna] < limite_inferior, coluna] = np.nan
    df.loc[df[coluna] > limite_superior, coluna] = np.nan

  return df


In [ ]:
# Exemplo de uso
df = pd.DataFrame({
  "coluna1": [1, 2, 3, 4, 5, 10],
  "coluna2": [10, np.nan, 30, 40, 50, 65],
})



In [ ]:
cols = df.columns

cols

Index(['coluna1', 'coluna2'], dtype='object')

In [ ]:
df = df.to_numpy()

df

array([[ 1., 10.],
       [ 2., nan],
       [ 3., 30.],
       [ 4., 40.],
       [ 5., 50.],
       [10., 65.]])

In [ ]:
df = pd.DataFrame(df, columns=cols)

df

,coluna1,coluna2
0,1.0,10.0
1,2.0,NaN
2,3.0,30.0
3,4.0,40.0
4,5.0,50.0
5,10.0,65.0


In [ ]:
df_sem_outliers = remover_outliers_iqr(df.copy())

print(df_sem_outliers)

   coluna1  coluna2
0      1.0     10.0
1      2.0      NaN
2      3.0     30.0
3      4.0     40.0
4      5.0     50.0
5      NaN     65.0


In [ ]:
df

,coluna1,coluna2
0,1.0,10.0
1,2.0,20.0
2,3.0,30.0
3,4.0,40.0
4,5.0,50.0
5,NaN,60.0


In [21]:

# Algoritmo Python para verificar e preencher valores nulos em um DataFrame
# Python

import pandas as pd
import numpy as np

def preencher_valores_nulos(dataframe):
  """
  Verifica todas as colunas de um DataFrame e preenche valores nulos com 0.

  Argumentos:
    dataframe: O DataFrame a ser analisado.

  Retorna:
    O DataFrame original com os valores nulos preenchidos por 0.
  """

  # Verifica se existem colunas com 100% de valores nulos
  for coluna in dataframe.columns:
    if dataframe[coluna].isnull().all():
      # Preenche os valores nulos com 0
      dataframe[coluna].fillna(0, inplace=True)

  return dataframe

# Exemplo de uso
dataframe_original = pd.DataFrame({
  'Coluna 1': [np.nan, np.nan, np.nan],
  'Coluna 2': [4, 5, 6],
  'Coluna 3': [np.nan, np.nan, np.nan]
})

print("DataFrame original:")
print(dataframe_original)

dataframe_modificado = preencher_valores_nulos(dataframe_original.copy())

print("\nDataFrame modificado:")
print(dataframe_modificado)

DataFrame original:
   Coluna 1  Coluna 2  Coluna 3
0       NaN         4       NaN
1       NaN         5       NaN
2       NaN         6       NaN

DataFrame modificado:
   Coluna 1  Coluna 2  Coluna 3
0       0.0         4       0.0
1       0.0         5       0.0
2       0.0         6       0.0


In [1]:
import mdatagen

print(mdatagen.__version__) 


0.0.91
